## Optical Flow

In [168]:
from datetime import datetime

from lib import loadSeries

## Read the Radar Data

In [169]:
## Set Date to Read Files
date = datetime.strptime("20140103234809", "%Y%m%d%H%M%S")

In [249]:
R = loadSeries.readFiles(date)

File Loaded at times : 9 sbmn_rain_rates_20140103_220011.nc
File Loaded at times : 8 sbmn_rain_rates_20140103_221211.nc
File Loaded at times : 7 sbmn_rain_rates_20140103_222412.nc
File Loaded at times : 6 sbmn_rain_rates_20140103_223612.nc
File Loaded at times : 5 sbmn_rain_rates_20140103_224810.nc
File Loaded at times : 4 sbmn_rain_rates_20140103_230011.nc
File Loaded at times : 3 sbmn_rain_rates_20140103_231210.nc
File Loaded at times : 2 sbmn_rain_rates_20140103_232410.nc
File Loaded at times : 1 sbmn_rain_rates_20140103_233610.nc
File Loaded at times : 0 sbmn_rain_rates_20140103_234809.nc


In [250]:
## APPLY OPTICAL FLOW FOR 3 LAST
R[-3:, :, :]

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])

In [279]:
import numpy as np
from numpy.ma.core import MaskedArray

def optical_flow(input_images):

    
    input_images = input_images.copy()
    
    nr_fields = input_images.shape[0]
    domain_size = (input_images.shape[1], input_images.shape[2])
    
    
    xy = np.empty(shape=(0, 2))
    uv = np.empty(shape=(0, 2))
    
    for n in range(nr_fields - 1):
        prvs_img = input_images[n, :, :].copy()
        next_img = input_images[n + 1, :, :].copy()
        
        if ~isinstance(prvs_img, MaskedArray):
            prvs_img = np.ma.masked_invalid(prvs_img)
            np.ma.set_fill_value(prvs_img, prvs_img.min())
            
        if ~isinstance(next_img, MaskedArray):
            next_img = np.ma.masked_invalid(next_img)
            np.ma.set_fill_value(next_img, next_img.min())
            
           
        # features detection
        points = ShiTomasi_detection(prvs_img)
    
        # get sparse u, v vectors with Lucas-Kanade tracking
        xy_, uv_ = lukasKanade(prvs_img, next_img, points)
        
        xy = np.append(xy, xy_, axis=0)

In [276]:
import cv2

def ShiTomasi_detection(input_image, max_corners=500, quality_level=0.1,
                        min_distance=3, block_size=15, buffer_mask=0,
                        use_harris = False, k = 0.04):
    
    input_image = np.copy(input_image)
    
    if input_image.ndim != 2:
        raise ValueError("input_image must be a two-dimensional array")
        
    # masked array
    if ~isinstance(input_image, MaskedArray):
        input_image = np.ma.masked_invalid(input_image)
    np.ma.set_fill_value(input_image, input_image.min())
    
    # buffer the quality mask to ensure that no vectors are computed nearby
    # the edges of the radar mask
    mask = np.ma.getmaskarray(input_image).astype("uint8")
    if buffer_mask > 0:
        mask = cv2.dilate(
            mask, np.ones((int(buffer_mask), int(buffer_mask)), np.uint8), 1
        )
        input_image[mask] = np.ma.masked

    # scale image between 0 and 255
    input_image = (
        (input_image.filled() - input_image.min())
        / (input_image.max() - input_image.min())
        * 255
    )
    
    # convert to 8-bit
    input_image = np.ndarray.astype(input_image, "uint8")
    mask = (-1 * mask + 1).astype("uint8")
    
    
    params = dict(
        maxCorners=max_corners,
        qualityLevel=quality_level,
        minDistance=min_distance,
        useHarrisDetector=use_harris,
        k=k,
    )
    
    points = cv2.goodFeaturesToTrack(input_image, mask=mask, **params)
    
    return points

In [281]:
def lukasKanade(prvs_image,next_image,points,
        winsize=(50, 50),
        nr_levels=3,
        criteria=(3, 10, 0),
        flags=0,
        min_eig_thr=1e-4):
    
    prvs_img = np.copy(prvs_image)
    next_img = np.copy(next_image)
    p0 = np.copy(points)
    
    if ~isinstance(prvs_img, MaskedArray):
        prvs_img = np.ma.masked_invalid(prvs_img)
    np.ma.set_fill_value(prvs_img, prvs_img.min())

    if ~isinstance(next_img, MaskedArray):
        next_img = np.ma.masked_invalid(next_img)
    np.ma.set_fill_value(next_img, next_img.min())
    
    # scale between 0 and 255
    prvs_img = ((prvs_img.filled() - prvs_img.min()) /
                (prvs_img.max() - prvs_img.min()) * 255)

    next_img = ((next_img.filled() - next_img.min()) /
                (next_img.max() - next_img.min()) * 255)

    # convert to 8-bit
    prvs_img = np.ndarray.astype(prvs_img, "uint8")
    next_img = np.ndarray.astype(next_img, "uint8")

    # Lucas-Kanade
    # TODO: use the error returned by the OpenCV routine
    params = dict(
        winSize=winsize,
        maxLevel=nr_levels,
        criteria=criteria,
        flags=flags,
        minEigThreshold=min_eig_thr,
    )
    p1, st, __ = cv2.calcOpticalFlowPyrLK(prvs_img, next_img,
                                          p0, None, **params)
    
    # keep only features that have been found
    st = st.squeeze() == 1
    
    if np.any(st):
        p1 = p1[st, :]
        p0 = p0[st, :]

        # extract vectors
        xy = p0
        uv = p1 - p0
        
        print(xy)
    
    return xy, uv

In [282]:
r = optical_flow(R[-3:, :, :])

[[[ 77. 101.]]

 [[ 59. 132.]]

 [[ 59. 135.]]

 [[ 73.  98.]]]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [163]:
r